In [2]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor, sarcopenia, sleepdisturb, chrpain, iatrog, constipation, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [46]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [5]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]

# Sindromes geriatricos

## Indice de Charslton

## Caidas previas

In [6]:
caprev_CON_PA= (CON_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_CON_PA:.2f}%")

Porcentaje de caidas previas: 5.88%


# Historial medico/comorbididades